In [2]:
using Pkg, Revise
Pkg.activate(ENV["JULIA_DQMC"])
include(joinpath(ENV["JULIA_DQMC"], "src/dqmc_framework.jl"))


function mc_from_inxml(inxml::AbstractString)
    Random.seed!(1234)
    if SUPPRESS_DQMC_INIT_OUTPUT
        old_stdout = stdout
        (rp, wp) = redirect_stdout()
    end

    @assert isfile(inxml)
    p = Params()
    p.output_file = "$(inxml[1:end-7]).out.h5.running"
    xml2parameters!(p, inxml)
    mc = DQMC(p)
    init!(mc)

    if SUPPRESS_DQMC_INIT_OUTPUT
        redirect_stdout(old_stdout)
        close(wp); close(rp);
    end
    return mc
end

(@isdefined SUPPRESS_DQMC_INIT_OUTPUT) || (const SUPPRESS_DQMC_INIT_OUTPUT = false)

false

# Run all tests

In [3]:
# const SUPPRESS_DQMC_INIT_OUTPUT = true
include("runtests.jl")

Running tests on thinkable.
L = 4
SLICES = 10
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_EVERY_NTH = 10
OPDIM = 3

CB = CBAssaad
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:/Users/carsten/Desktop/sciebo/lattices/square_L_4_W_4.xml"


Loading lattice with 16 sites
Initializing neighbor-tables
Initializing Peirls phases (Bfield)
Initializing hopping exponentials (Bfield)
Initializing hopping exponentials (Bfield, Checkerboard)
Checkerboard (Bfield) - exact (abs):		0.0024417942043431936

Initializing HS field
Initializing boson action

Building stack
Initial propagate: 11 -1


┌ Warning: Calling somewhat inefficient and potentially inaccurate `inv_one_plus_udt`
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:170
┌ Warning: Calling potentially inaccurate `inv_one_plus_udt!`
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:204
┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:604
┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:604
┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:604
┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:604
┌ Warning: Calling somewhat inefficient and potentially inaccurate `inv_sum_udts`
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\

L = 4
SLICES = 10
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_EVERY_NTH = 10
OPDIM = 3

CB = CBAssaad
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:/Users/carsten/Desktop/sciebo/lattices/square_L_4_W_4.xml"


Loading lattice with 16 sites
Initializing neighbor-tables
Initializing Peirls phases (Bfield)
Initializing hopping exponentials (Bfield)
Initializing hopping exponentials (Bfield, Checkerboard)
Checkerboard (Bfield) - exact (abs):		0.0024417942043431936

Initializing HS field
Initializing boson action

Building stack
Initial propagate: 11 -1
L = 5
SLICES = 10
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_

# Edit area

In [39]:
includet(joinpath(ENV["JULIA_DQMC"], "test/deps/ed.jl"))

In [196]:
function concat_greens_k_space(gkx, gky)
    Gkx = Diagonal(gkx[:])
    Gky = Diagonal(gky[:])
    # gk = cat(Gkx, Gky, Gkx, Gky, dims=(1,2)); # Full 4Nx4N momentum-space Green's function
    gk = cat(Gkx, Gky, dims=(1,2)); # 2Nx2N momentum-space Green's function
end

"""
    perform_ed_k_space(L = 4, beta = 8.) -> obs

Perform ED for L=2 system in momentum-space single mode basis.

Returns a couple of observable `obs` as a `Dict{String, Any}`.
"""
function perform_ed_k_space(; L::Integer = 4, beta::Float64 = 8.)
    N = L^2
    params_x = Dict("th" => 1.0, "tv" => 0.5, "mu" => -0.5)
    params_y = Dict("th" => -0.5, "tv" => -1.0, "mu" => -0.5)
    obs = Dict{String, Any}()

    # ETGF
    gkx = ifftshift(GF_k_space(params_x, L, beta));
    gky = ifftshift(GF_k_space(params_y, L, beta));

    gk = concat_greens_k_space(gkx, gky); # Full momentum-space Green's function

    g = ifft_greens(gk, flv=2)
    @assert maximum(imag(g)) < 1e-12
    obs["greens"] = real(g) # Full (4xN, 4xN) real-space Green's function



    # TDGF
    taus = collect(0.0:0.1:beta)[1:end-1]

    # Gt0
    gt0_k_x = [TDGF_k_space(params_x, L, beta, tau) for tau in taus]
    gt0_k_y = [TDGF_k_space(params_y, L, beta, tau) for tau in taus]
    gt0_k = concat_greens_k_space.(gt0_k_x, gt0_k_y)
    obs["Gt0"] = real.(ifft_greens.(gt0_k; ifftshift=true, flv=2));
    
    # G0t
    # TODO: TDGF_k_space_G0t
    g0t_k_x = [TDGF_k_space(params_x, L, beta, tau; G0t=true) for tau in taus]
    g0t_k_y = [TDGF_k_space(params_y, L, beta, tau; G0t=true) for tau in taus]
    g0t_k = concat_greens_k_space.(g0t_k_x, g0t_k_y)
    obs["G0t"] = real.(ifft_greens.(g0t_k; ifftshift=true, flv=2));

    return obs
end
edk = perform_ed_k_space(L=4, beta=10.)

Dict{String,Any} with 3 entries:
  "G0t"    => Array{Float64,2}[[-0.375418 -0.124582 … 0.0 0.0; -0.124582 -0.375…
  "greens" => [0.624582 -0.124582 … 0.0 0.0; -0.124582 0.624582 … 0.0 0.0; … ; …
  "Gt0"    => Array{Float64,2}[[0.624582 -0.124582 … 0.0 0.0; -0.124582 0.62458…

In [199]:
maximum.(mc_edk.s.meas.G0t - edk["G0t"])

100-element Array{Float64,1}:
 4.829470157119431e-15 
 4.218847493575595e-15 
 3.7192471324942744e-15
 3.3584246494910985e-15
 3.58046925441613e-15  
 3.635980405647388e-15 
 3.6914915568786455e-15
 3.83026943495679e-15  
 4.3021142204224816e-15
 5.023759186428833e-15 
 2.761679773755077e-15 
 2.7478019859472624e-15
 2.706168622523819e-15 
 ⋮                     
 3.70536934468646e-15  
 3.8719027983802334e-15
 3.6637359812630166e-15
 3.3306690738754696e-15
 2.914335439641036e-15 
 2.9976021664879227e-15
 2.942091015256665e-15 
 2.7200464103316335e-15
 2.6645352591003757e-15
 2.9698465908722937e-15
 3.608224830031759e-15 
 3.885780586188048e-15 

In [202]:
@testset "Compare to momentum-space ED" begin
    L = mc_edk.p.L
    N = mc_edk.l.sites

    # ETGF
    @test isapprox(mc_edk.s.greens, edk["greens"])
    
    
    # TDGFs: Gt0, G0t
    allocate_tdgfs!(mc_edk)
    calc_tdgfs!(mc_edk)
    
    # Gt0
    f = () -> begin
        for i in 1:mc_edk.p.slices
            isapprox(mc_edk.s.meas.Gt0[i], edk["Gt0"][i]) || return false
        end
        return true
    end
    @test f()
  
    # G0t
    f = () -> begin
        for i in 1:mc_edk.p.slices
            isapprox(mc_edk.s.meas.G0t[i], edk["G0t"][i]) || return false
        end
        return true
    end
    @test f()
end

Allocated memory for TDGF measurement.
Test Summary:                | Pass  Total
Compare to momentum-space ED |    3      3


Test.DefaultTestSet("Compare to momentum-space ED", Any[], 3, false)

# Run subset of tests

In [204]:
include("tests_freefermions.jl")

L = 2
SLICES = 80
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 1
CHECKERBOARD = FALSE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.0
U = 0.0
R = 1.0
C = 0.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = FALSE
WRITE_EVERY_NTH = 10
OPDIM = 1

CB = CBFalse
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:/Users/carsten/Desktop/sciebo/lattices/square_L_2_W_2.xml"


Loading lattice with 4 sites
Initializing neighbor-tables
Initializing hopping exponentials

Initializing HS field
Initializing boson action

Building stack
Initial propagate: 81 -1
Performing real-space ED...
Done with ED.
L = 4
SLICES = 100
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = FALSE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.0
U = 0.0
R = 1.0
C = 0.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = FALSE
WRITE_EVERY_NTH = 10
OPDIM = 1

CB = CBFalse
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.l